## Import all Necessary libraries

In [151]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer as tf_idf
from sklearn.metrics.pairwise import cosine_similarity
import pickle

In [152]:
import nltk
nltk.data.path.append('C:/Users/patyu/AppData/Roaming/nltk_data')

from nltk.corpus import wordnet
from nltk.stem.wordnet import WordNetLemmatizer

nltk.download('averaged_perceptron_tagger', download_dir='C:/Users/patyu/AppData/Roaming/nltk_data')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:/Users/patyu/AppData/Roaming/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [153]:
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
print(lemmatizer.lemmatize('running'))  # Should print 'running'

running


In [154]:
movies=pd.read_csv('netflix_titles.csv') #read in the netflix file

In [155]:
movies.shape #dataset has seven columns, and 8807 movies(rows)

(8807, 12)

In [156]:
movies.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...


## Preprocessing Begins

In [157]:
#list of columns that will be needed
#show_id
#type
#title
#cast
#country
#listed_in
#description

In [158]:
#drop unnecessary columns
movies.drop("date_added", axis=1, inplace=True)
movies.drop("release_year", axis=1, inplace=True)
movies.drop("duration", axis=1, inplace=True)
movies.drop("rating", axis=1, inplace=True)
movies.drop('director',axis=1, inplace=True)

In [159]:
movies.isnull().sum() #checking for null values in needed columns

show_id          0
type             0
title            0
cast           825
country        831
listed_in        0
description      0
dtype: int64

In [160]:
movies=movies.dropna() #dropping missing values

In [161]:
movies.isnull().sum()#checking to make sure there are no null values anymore

show_id        0
type           0
title          0
cast           0
country        0
listed_in      0
description    0
dtype: int64

In [162]:
movies.shape 

(7305, 7)

In [163]:
print(movies.iloc[0]['cast'])
print(movies.iloc[0]['listed_in'])
print(movies.iloc[0]['country'])
print(movies.iloc[0]['description'])
print(movies.iloc[0]['type'])

Ama Qamata, Khosi Ngema, Gail Mabalane, Thabang Molaba, Dillon Windvogel, Natasha Thahane, Arno Greeff, Xolile Tshabalala, Getmore Sithole, Cindy Mahlangu, Ryle De Morny, Greteli Fincham, Sello Maake Ka-Ncube, Odwa Gwanya, Mekaila Mathys, Sandi Schultz, Duane Williams, Shamilla Miller, Patrick Mofokeng
International TV Shows, TV Dramas, TV Mysteries
South Africa
After crossing paths at a party, a Cape Town teen sets out to prove whether a private-school swimming star is her sister who was abducted at birth.
TV Show


Since the columns are in string formats and are needed in the list data structure, I'll apply split to each of the columns so it returns list. 

In [164]:
movies['cast']=movies['cast'].apply(lambda x:x.split(',')) 
movies['listed_in']=movies['listed_in'].apply(lambda x:x.split(',')) 
movies['country']=movies['country'].apply(lambda x:x.split(','))
movies['description']=movies['description'].apply(lambda x:x.split())
movies['type']=movies['type'].apply(lambda x:x.split(','))

In [165]:
print(movies.iloc[0]['cast'])
print(movies.iloc[0]['listed_in'])
print(movies.iloc[0]['country'])
print(movies.iloc[0]['description'])
print(movies.iloc[0]['type'])

['Ama Qamata', ' Khosi Ngema', ' Gail Mabalane', ' Thabang Molaba', ' Dillon Windvogel', ' Natasha Thahane', ' Arno Greeff', ' Xolile Tshabalala', ' Getmore Sithole', ' Cindy Mahlangu', ' Ryle De Morny', ' Greteli Fincham', ' Sello Maake Ka-Ncube', ' Odwa Gwanya', ' Mekaila Mathys', ' Sandi Schultz', ' Duane Williams', ' Shamilla Miller', ' Patrick Mofokeng']
['International TV Shows', ' TV Dramas', ' TV Mysteries']
['South Africa']
['After', 'crossing', 'paths', 'at', 'a', 'party,', 'a', 'Cape', 'Town', 'teen', 'sets', 'out', 'to', 'prove', 'whether', 'a', 'private-school', 'swimming', 'star', 'is', 'her', 'sister', 'who', 'was', 'abducted', 'at', 'birth.']
['TV Show']


In [166]:
#create a function to get the first three names of the cast members
def get_three_people(obj):
    three_people=[]
    counter=0
    for i in (obj):
        if counter != 3:
            three_people.append(i)
            counter+=1
        else:
            break
    return three_people

In [167]:
movies['cast']=movies['cast'].apply(get_three_people)

In [168]:
#renaming the columns to look better
movies=movies.rename(columns={'type':'Type','title':'Title','cast':'Cast','country':'Country','listed_in':'Genre','description':'Overview'})

In [169]:
movies.head() #Now we have it how we want but there are still some works to do

,show_id,Type,Title,Cast,Country,Genre,Overview
1,s2,[TV Show],Blood & Water,"[Ama Qamata, Khosi Ngema, Gail Mabalane]",[South Africa],"[International TV Shows, TV Dramas, TV Myste...","[After, crossing, paths, at, a, party,, a, Cap..."
4,s5,[TV Show],Kota Factory,"[Mayur More, Jitendra Kumar, Ranjan Raj]",[India],"[International TV Shows, Romantic TV Shows, ...","[In, a, city, of, coaching, centers, known, to..."
7,s8,[Movie],Sankofa,"[Kofi Ghanaba, Oyafunmike Ogunlano, Alexandr...","[United States, Ghana, Burkina Faso, United...","[Dramas, Independent Movies, International M...","[On, a, photo, shoot, in, Ghana,, an, American..."
8,s9,[TV Show],The Great British Baking Show,"[Mel Giedroyc, Sue Perkins, Mary Berry]",[United Kingdom],"[British TV Shows, Reality TV]","[A, talented, batch, of, amateur, bakers, face..."
9,s10,[Movie],The Starling,"[Melissa McCarthy, Chris O'Dowd, Kevin Kline]",[United States],"[Comedies, Dramas]","[A, woman, adjusting, to, life, after, a, loss..."


Since each of the rows are now in list format, I need to remove spaces between each words. So, 'Mary Berry' becomes 'MaryBerry'.
This creates uniformity

In [170]:
movies['Genre']=movies['Genre'].apply(lambda x: [i.replace(" ","") for i in x]) 
movies['Cast']=movies['Cast'].apply(lambda x: [i.replace(" ","") for i in x])
movies['Country']=movies['Country'].apply(lambda x: [i.replace(" ","") for i in x])
movies['Type']=movies['Type'].apply(lambda x: [i.replace(" ","") for i in x])

In [171]:
#Now add all the useful features under a new column called Tag
movies['Tags']=movies['Cast']+movies['Country']+movies['Genre']+movies['Overview']+movies['Type']

In [172]:
Netflix_df=movies[['show_id','Title','Tags']] #creating a new dataframe with the new important columns

In [173]:
Netflix_df.head()

,show_id,Title,Tags
1,s2,Blood & Water,"[AmaQamata, KhosiNgema, GailMabalane, SouthAfr..."
4,s5,Kota Factory,"[MayurMore, JitendraKumar, RanjanRaj, India, I..."
7,s8,Sankofa,"[KofiGhanaba, OyafunmikeOgunlano, AlexandraDua..."
8,s9,The Great British Baking Show,"[MelGiedroyc, SuePerkins, MaryBerry, UnitedKin..."
9,s10,The Starling,"[MelissaMcCarthy, ChrisO'Dowd, KevinKline, Uni..."


In [174]:
Netflix_df['Tags']=Netflix_df['Tags'].apply(lambda x:" ".join(x)) #converting back to string

C:\Users\patyu\AppData\Local\Temp\ipykernel_17488\2229422181.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Netflix_df['Tags']=Netflix_df['Tags'].apply(lambda x:" ".join(x)) #converting back to string


In [175]:
Netflix_df=Netflix_df.reset_index() #resets index

In [176]:
Netflix_df.head()

,index,show_id,Title,Tags
0,1,s2,Blood & Water,AmaQamata KhosiNgema GailMabalane SouthAfrica ...
1,4,s5,Kota Factory,MayurMore JitendraKumar RanjanRaj India Intern...
2,7,s8,Sankofa,KofiGhanaba OyafunmikeOgunlano AlexandraDuah U...
3,8,s9,The Great British Baking Show,MelGiedroyc SuePerkins MaryBerry UnitedKingdom...
4,9,s10,The Starling,MelissaMcCarthy ChrisO'Dowd KevinKline UnitedS...


Creating a function that extracts the part of speech of a word and put it in the format a lemmatizer would take. 

In [177]:
def get_wordnet_pos(word):
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)

Now to lemmatize each of the words to the root word while also considering context.

In [178]:
print(nltk.data.path)

['C:\\Users\\patyu/nltk_data', 'c:\\Users\\patyu\\OneDrive\\Documents\\GitHub\\Netflix-movie-recommender\\venv\\nltk_data', 'c:\\Users\\patyu\\OneDrive\\Documents\\GitHub\\Netflix-movie-recommender\\venv\\share\\nltk_data', 'c:\\Users\\patyu\\OneDrive\\Documents\\GitHub\\Netflix-movie-recommender\\venv\\lib\\nltk_data', 'C:\\Users\\patyu\\AppData\\Roaming\\nltk_data', 'C:\\nltk_data', 'D:\\nltk_data', 'E:\\nltk_data', 'C:/Users/patyu/AppData/Roaming/nltk_data', 'C:/Users/patyu/AppData/Roaming/nltk_data', 'C:/Users/patyu/AppData/Roaming/nltk_data', 'C:/Users/patyu/AppData/Roaming/nltk_data']


In [179]:
def lemmatizer(word):
    lemmatizer=WordNetLemmatizer()
    return lemmatizer.lemmatize(word,get_wordnet_pos(word))

In [180]:
Netflix_df['Tags']=Netflix_df['Tags'].apply(lemmatizer) 

A tf_idf vectorizer is used to convert words to vectors based on their frequency and importance.Fit transform will basically fit the words into what we've set for tfidf and the toarray returns an array of vectors

In [181]:
vectorizer=tf_idf(max_features=4000,stop_words='english',lowercase=True) 

In [182]:
vectors=vectorizer.fit_transform(Netflix_df['Tags']).toarray() 

In [183]:
vectors.shape #this shows for 7305 movies, there are 4000 features(words) each.

(7305, 4000)

In [184]:
vectorizer.get_feature_names_out()#this shows each of the features. 

array(['000', '10', '100', ..., 'zone', 'zooeydeschanel',
       'álvarocervantes'], dtype=object)

## and that's all with preprocessing. 

The cosine similarity will measure the similarity between each feature words and movies

In [185]:
similarity=cosine_similarity(vectors) 

In [186]:
similarity

array([[1.        , 0.02867151, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.02867151, 1.        , 0.        , ..., 0.04722015, 0.13499842,
        0.03535464],
       [0.        , 0.        , 1.        , ..., 0.00821072, 0.01102847,
        0.05684675],
       ...,
       [0.        , 0.04722015, 0.00821072, ..., 1.        , 0.02517024,
        0.00296643],
       [0.        , 0.13499842, 0.01102847, ..., 0.02517024, 1.        ,
        0.00398444],
       [0.        , 0.03535464, 0.05684675, ..., 0.00296643, 0.00398444,
        1.        ]])

In [187]:
sorted(similarity[3],reverse=True) #this shows the cosine similarity of the 4th movie

[1.0000000000000002,
 0.31121031669321914,
 0.26380957093507507,
 0.24026918080346102,
 0.2215874875114165,
 0.21111382430765355,
 0.20995880642919362,
 0.2098510315380286,
 0.20653613333899473,
 0.2065344024186292,
 0.20145524617513053,
 0.20072695538249208,
 0.19983458979144805,
 0.1931789370317451,
 0.18649281021539443,
 0.18585110987396727,
 0.1833087516586945,
 0.17622324430298633,
 0.16815523102046412,
 0.16761727704308982,
 0.1660834931480766,
 0.16557717675312697,
 0.16313391349080528,
 0.16297587512515094,
 0.1599047309956229,
 0.15743486790471942,
 0.15416730128807352,
 0.15192369095039202,
 0.14791074464631437,
 0.1467979723764508,
 0.1465380088866382,
 0.1453761361994394,
 0.14495578105225962,
 0.14480217251485625,
 0.14379556971524182,
 0.14374534999105326,
 0.14371027468373113,
 0.14365656358209217,
 0.14332314409419733,
 0.14278027413697547,
 0.1391206034027392,
 0.1372171857744054,
 0.13651701363403188,
 0.1347669316186243,
 0.1301145757274201,
 0.1300696143607941,
 0.1

Now is time to write the recommendation function. The function takes a Title argument and checks with the dataset where the index of the title falls. it uses the index to bring a list of the movie's similarity score(compared to other movies). The scores are arranged in descending order and the first 10 are sliced. The Titles are then retrieved from the dataset. 

In [188]:
def recommendation(Title): 
    movie_index=Netflix_df[Netflix_df['Title']==Title].index[0] 
    distances=similarity[movie_index] 
    movies_list=sorted(list(enumerate(distances)),reverse=True,key=lambda x:x[1])[1:10]
    for i in movies_list:
        print(Netflix_df.iloc[i[0]].Title) 

In [189]:
recommendation("Khoobsurat") #up and working!!

Dil Hai Tumhaara
A Beautiful Life
Hole in the Wall
Finally Found Someone
Hunterrr
Thank You
Bhaage Re Mann
Disenchantment
Manje Bistre


In [190]:
pickle.dump(Netflix_df.to_dict(),open('Netflix_dict.pkl','wb')) #Now to deploy locally. Use pickle to dump files so that you
#can load them later. 

In [191]:
similarities=pickle.dump(similarity,open('similarity.pkl','wb'))

## The END